# Merge individual sweeps
* author: [Hamid Ali Syed](https://twitter.com/hamidrixvi)
* date: Dec 23, 2022

This data is from three mobile radars, COW1, DOW7, and DOW8, which belong to the University of Illinois. The data is packed in a way that each sweep radar has scanned makes one file, i.e., (one file for one tilt). However, we want it to be in Cfradial format (why?).<p>
Cfradial is a data format developed by the National Center for Atmospheric Research (NCAR) for storing radar data. It is based on the NetCDF (Network Common Data Form) data format, a self-describing data format that allows for storing multidimensional scientific data. The Cfradial format is used to store radar data collected in a volume scan, which includes multiple sweeps of the radar at different tilts. It is a widely used format for storing and exchanging radar data, and many radar data processing and analysis software packages support it.

## Instructions:
**Start by importing the necessary packages:**
1. Use the `warnings` package to ignore any warnings that may appear during the execution of your code.
2. Use the `%%time` magic command to measure the execution time of your code.
3. Use the `numpy` package to perform numerical operations on arrays.
4. Use the `glob` package to search for files that match a specific pattern.
5. Use the `os` and `sys` packages to interact with the operating system and access system-specific parameters and functions.
6. Use the `shutil` package to perform high-level file operations, such as copying and deleting files.

In [1]:
%%time
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import glob
import os, sys
import shutil

CPU times: user 45.6 ms, sys: 25.6 ms, total: 71.2 ms
Wall time: 205 ms


In [2]:
basedir = "/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/IOP3/"

In [3]:
data_dir = ['COW1', "DOW7", "DOW8"]

In [4]:
def apply_func(data_dir):
    curr_dir_list = []
    for path in data_dir:
        files = sorted(glob.glob(os.path.join(basedir, path, "*SUR.nc")))
        print(f"{path}: {len(files)}")
        # force the if statement below to be True on first run
        el = 99999999
        for f in files:
            new_el = float(f.split("_")[-2].replace("el", ""))
            if new_el < el:
                # store new dir name
                curr_dir = f.split(".")[1]
                curr_dir_list.append(os.path.join(basedir, path, curr_dir))
                # create directory
                os.makedirs(os.path.join(basedir, path, curr_dir), exist_ok=True)
#             store new el
            el = new_el
#             print(f"{basedir}{os.sep}{path}{os.sep}{curr_dir}{os.sep}{f.split(os.sep)[-1]}")
#             move file
            shutil.copy(f"{f}", f"{basedir}{os.sep}{path}{os.sep}{curr_dir}{os.sep}{f.split(os.sep)[-1]}")
    return curr_dir_list

In [5]:
curr_dir_list = apply_func(data_dir)

COW1: 1160
DOW7: 918
DOW8: 1177


In [6]:
for new_files in curr_dir_list[:2]:
    print(os.path.join(new_files.split("202204")[0], "merged"))

/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/IOP3/COW1/merged
/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/IOP3/COW1/merged


In [7]:
new_path = os.path.join(new_files.split("202204")[0], "merged")
new_path

'/depot/dawson29/data/Projects/PERiLS/obsdata/2022/Illinois_Mobile_Radar/IOP3/COW1/merged'

In [8]:
mod_cmd = "module use /depot/dawson29/data/Projects/PERiLS/modulefiles/ && module load lrose"

In [9]:
os.system(mod_cmd+" && module list")


Currently Loaded Modules:
  1) xalt/1.1.2            (S)   5) mpc/1.1.0          9) jupyterhub/2.0.0
  2) anaconda/2020.02-py37       6) zlib/1.2.11       10) lrose/20220222
  3) gmp/6.1.2                   7) gcc/9.3.0
  4) mpfr/3.1.6                  8) texlive/20200406

  Where:
   S:  Module is Sticky, requires --force to unload or purge

 



0

In [ ]:
for new_files in curr_dir_list:
    os.system(mod_cmd+" && RadxConvert -ag_all -f "+new_files+"/*"+ " -outdir "+new_path)

The data is being merged sucessfully and we can now perform quality check and other things in another notebook